# Haversine Formula vs Mercator Projection

## Written by Michael George

Simple comparison of two methods to calculate the distance between two points on a sphere:

1. Haversine formula is required over longer distances (e.g. navigation over hundreds of kilometres).
2. Mercator projection + Pythagorean theorem to produce quicker estimates over shorter distances (e.g. alpha proximity 50m).

In [1]:
from math import pi, radians, cos, sin, asin, atan2, sqrt

## Constants

Earth's [mean radius](https://en.wikipedia.org/wiki/Earth_radius#Published_values) was also taken from Wikipedia - IUGG and IERS both give a vlue of 6371008.7714 metres.

In [2]:
earthRadius = 6371009
earthCircum = 2 * pi * earthRadius
distPerDegLat = earthCircum / 360

## Unit Conversions

In [3]:
def semicirclesToDegrees(value):
    '''Convert semicircles to degrees'''

    return value * (180.0 / 2 ** 31)


def semicirclesToRadians(value):
    '''Convert semicircles to radians'''

    return radians(value * (180.0 / 2 ** 31))


def metresToKilometers(value):
    '''Convert metres to kilometers'''

    return value / 1000


def metresToMiles(value):
    '''Convert metres to kilometers'''

    return value / 1609.344

## Hathersine Formula

A popular approach when calculating the distance between two points is to use the Hathersine formula.

This requires the use of several math functions (e.g. 2 * sin, 2 * cos, 1 * asin, 1 * sqrt) with little opportunity to optimise.

Full details about the Haversine formula can be found on [Wikipedia](https://en.wikipedia.org/wiki/Haversine_formula).

In [4]:
def calculateDistanceRadians(latitude1r, longitude1r, latitude2r, longitude2r, radius=earthRadius):
    '''Calculate the great-circle distance between two points on a sphere given their longitudes and latitudes'''
    
    cosineCalc = cos(latitude1r) * cos(latitude2r)
    haversine = sin((latitude2r - latitude1r) / 2) ** 2 + cosineCalc * sin((longitude2r - longitude1r) / 2) ** 2
    
    # Alternative hathersine formula avoids cos() by replacing it with a more "expensive" calculation
    #sineCalc = 1 - sin((latitude1 - latitude2) / 2) ** 2 - sin((latitude1 + latitude2) / 2) ** 2
    #haversine = sin((latitude2 - latitude1) / 2) ** 2 + sineCalc * sin((longitude2 - longitude1) / 2) ** 2

    distance = 2 * radius * asin(sqrt(haversine))
    
    # Alternative distance calculation produces the same result and was only tested out of curiosity
    #distance = 2 * radius * atan2(sqrt(haversine), sqrt(1 - haversine))
    
    return distance


def calculateDistanceDegrees(latitude1d, longitude1d, latitude2d, longitude2d, radius=earthRadius):
    '''Calculate the great-circle distance between two points on a sphere given their longitudes and latitudes'''
    
    latitude1, longitude1, latitude2, longitude2 = map(radians, [latitude1d, longitude1d, latitude2d, longitude2d])
    
    return calculateDistanceRadians(latitude1, longitude1, latitude2, longitude2, radius=radius)


def calculateDistanceSemicircles(latitude1c, longitude1c, latitude2c, longitude2c, radius=earthRadius):
    '''Calculate the great-circle distance between two points on a sphere given their longitudes and latitudes'''
    
    latitude1, longitude1, latitude2, longitude2 = map(semicirclesToRadians, [latitude1c, longitude1c, latitude2c, longitude2c])

    return calculateDistanceRadians(latitude1, longitude1, latitude2, longitude2, radius=radius)

## Mercator Projection + Pythagorean Theorem

A quicker way approximate the distance between two points is to map corrdinates to a 2D grid and use the Pythagorean theorum.

This requires far less math functions (1 * cos, 1 * sqrt) with few multiplications and divisions.

Details about Mercator projections can be found on [Wikipedia](https://en.wikipedia.org/wiki/Mercator_projection#Mathematics).

In [5]:
def estimateDistanceRadians(latitude1r, longitude1r, latitude2r, longitude2r, radius=earthRadius):
    '''Estimate the distance between two points on a sphere using a simple Mercator projection'''

    y1 = latitude1r * earthRadius
    y2 = latitude2r * earthRadius
    
    # Using a single scaling factor produces the most consistent estimate
    #distPerRadLon = cos(latitude1r) * earthRadius  
    #x1 = longitude1r * distPerRadLon
    #x2 = longitude2r * distPerRadLon
    
    # Can use this for performance (calculate once per point) but rounding errors can affect cm accuracy
    x1 = longitude1r * cos(latitude1r) * earthRadius  
    x2 = longitude2r * cos(latitude2r) * earthRadius  
    
    # Apply Pythagorean theorem to determine the length of the hypotenuse
    distance = sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

    return distance


def estimateDistanceDegrees(latitude1d, longitude1d, latitude2d, longitude2d, radius=earthRadius):
    '''Estimate the distance between two points on a sphere using a simple Mercator projection'''
    
    y1 = latitude1d * distPerDegLat
    y2 = latitude2d * distPerDegLat
    
    # Using a single scaling factor produces the most consistent estimate
    #distPerDegLon = cos(radians(latitude2d)) * distPerDegLat   
    #x1 = longitude1d * distPerDegLon
    #x2 = longitude2d * distPerDegLon

    # Can use this for performance (calculate once per point) but rounding errors can affect cm accuracy
    x1 = longitude1d * cos(radians(latitude1d)) * distPerDegLat
    x2 = longitude2d * cos(radians(latitude2d)) * distPerDegLat
    
    # Apply Pythagorean theorem to determine the length of the hypotenuse
    distance = sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

    return distance


def estimateDistanceSemicircles(latitude1c, longitude1c, latitude2c, longitude2c, radius=earthRadius):
    '''Estimate the distance between two points on a sphere using a simple Mercator projection'''
    
    latitude1, longitude1, latitude2, longitude2 = map(semicirclesToRadians, [latitude1c, longitude1c, latitude2c, longitude2c])

    return estimateDistanceRadians(latitude1, longitude1, latitude2, longitude2, radius=radius)

## Sanity Checks

In [6]:
# https://www.vcalc.com/wiki/vCalc/Haversine+-+Distance
radius1 = 6371009

# Expect 216.904 km - https://andrew.hedges.name/experiments/haversine/
radius2 = 6373000

In [7]:
stevenageLat = 51.9038
stevenageLon = -0.1966

portlandLat = 50.5475
portlandLon = -2.4343

# Expect 216.904 km using radius2
distance = calculateDistanceDegrees(stevenageLat, stevenageLon, portlandLat, portlandLon, radius=radius2)
print('Calculated distance (Haversine formula, radius {}) is approximately {:.3f} kilometres / {:.3f} miles'.format(
        radius2, metresToKilometers(distance), metresToMiles(distance)))

# Expect 216.837 km using default radius
distance = calculateDistanceDegrees(stevenageLat, stevenageLon, portlandLat, portlandLon, radius=earthRadius)
print('Calculated distance (Haversine formula, radius {}) is approximately {:.3f} kilometres / {:.3f} miles'.format(
        earthRadius, metresToKilometers(distance), metresToMiles(distance)))

# Expect 216.84 km using radius1
distance = calculateDistanceDegrees(stevenageLat, stevenageLon, portlandLat, portlandLon, radius=radius1)
print('Calculated distance (Haversine formula, radius {}) is approximately {:.2f} kilometres / {:.2f} miles'.format(
        radius1, metresToKilometers(distance), metresToMiles(distance)))

distance = estimateDistanceDegrees(stevenageLat, stevenageLon, portlandLat, portlandLon)
print('Estimated distance (Mercator projection) is approximately {:.2f} kilometres / {:.2f} miles'.format(
        metresToKilometers(distance), metresToMiles(distance)))

Calculated distance (Haversine formula, radius 6373000) is approximately 216.904 kilometres / 134.778 miles
Calculated distance (Haversine formula, radius 6371009) is approximately 216.837 kilometres / 134.736 miles
Calculated distance (Haversine formula, radius 6371009) is approximately 216.84 kilometres / 134.74 miles
Estimated distance (Mercator projection) is approximately 218.80 kilometres / 135.95 miles


In [8]:
stevenageLat = 51.9038
stevenageLon = -0.1966

wareLat = 51.8104
wareLon = -0.0282

# Expect 15.549 km using radius2
distance = calculateDistanceDegrees(stevenageLat, stevenageLon, wareLat, wareLon, radius=radius2)
print('Calculated distance (Haversine formula, radius {}) is approximately {:.3f} kilometres / {:.3f} miles'.format(
        radius2, metresToKilometers(distance), metresToMiles(distance)))

# Expect 15.544 km using default radius
distance = calculateDistanceDegrees(stevenageLat, stevenageLon, wareLat, wareLon, radius=earthRadius)
print('Calculated distance (Haversine formula, radius {}) is approximately {:.3f} kilometres / {:.3f} miles'.format(
        earthRadius, metresToKilometers(distance), metresToMiles(distance)))

# Expect 15.54 km using radius1
distance = calculateDistanceDegrees(stevenageLat, stevenageLon, wareLat, wareLon, radius=radius1)
print('Calculated distance (Haversine formula, radius {}) is approximately {:.2f} kilometres / {:.2f} miles'.format(
        radius1, metresToKilometers(distance), metresToMiles(distance)))

distance = estimateDistanceDegrees(stevenageLat, stevenageLon, wareLat, wareLon)
print('Estimated distance (Mercator projection) is approximately {:.2f} kilometres / {:.2f} miles'.format(
        metresToKilometers(distance), metresToMiles(distance)))

Calculated distance (Haversine formula, radius 6373000) is approximately 15.549 kilometres / 9.662 miles
Calculated distance (Haversine formula, radius 6371009) is approximately 15.544 kilometres / 9.659 miles
Calculated distance (Haversine formula, radius 6371009) is approximately 15.54 kilometres / 9.66 miles
Estimated distance (Mercator projection) is approximately 15.53 kilometres / 9.65 miles


In [9]:
startLat = 620951524
startLon = -6830082

endLat = 620954030
endLon = -6823068

distance = calculateDistanceSemicircles(startLat, startLon, endLat, endLon)
print('Calculated distance (Haversine formula) is approximately {:.2f} metres'.format(distance))

distance = estimateDistanceSemicircles(startLat, startLon, endLat, endLon)
print('Estimated distance (Mercator projection) is approximately {:.2f} metres'.format(distance))

Calculated distance (Haversine formula) is approximately 46.50 metres
Estimated distance (Mercator projection) is approximately 46.66 metres


In [10]:
startLat = semicirclesToDegrees(startLat)
startLon = semicirclesToDegrees(startLon)

endLat = semicirclesToDegrees(endLat)
endLon = semicirclesToDegrees(endLon)

distance = calculateDistanceDegrees(startLat, startLon, endLat, endLon)
print('Calculated distance (Haversine formula) is approximately {:.2f} metres'.format(distance))

distance = estimateDistanceDegrees(startLat, startLon, endLat, endLon)
print('Estimated distance (Mercator projection) is approximately {:.2f} metres'.format(distance))

Calculated distance (Haversine formula) is approximately 46.50 metres
Estimated distance (Mercator projection) is approximately 46.66 metres
